In [1]:
from xgboost import XGBClassifier, XGBRegressor
from tensorflow import keras
import tensorflow as tf
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

In [2]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
train_df.head()

,Ticker,Company Name,Sector,Industry Group,Industry,Description,Sub-Industry
0,STAF,STAFFING 360 SOLUTIONS INC,Industrials,Commercial & Professional Services,Professional Services,"Staffing 360 Solutions, Inc. is an internation...",Human Resource & Employment Services
1,NXTC,NEXTCURE INC,Health Care,"Pharmaceuticals, Biotechnology & Life Sciences",Biotechnology,"NextCure, Inc. is a clinical-stage biopharmace...",Biotechnology
2,CVBF,CVB FINANCIAL CORP,Financials,Banks,Banks,CVB Financial Corp. is a bank holding company ...,Regional Banks
3,ARCT,ARCTURUS THERAPETCS HOLD INC,Health Care,"Pharmaceuticals, Biotechnology & Life Sciences",Biotechnology,Arcturus Therapeutics Holdings Inc. is a late-...,Biotechnology
4,SMPL,SIMPLY GOOD FOODS COMPANY,Consumer Staples,"Food, Beverage & Tobacco",Food Products,"Simply Good Foods Co is a developer, marketer ...",Packaged Foods & Meats


In [3]:
X = train_df.iloc[:, 2:-2]
y = train_df.iloc[:, -1]
test_df = test_df.iloc[:, 2:]
test_df

,Sector,Industry Group,Industry,Description
0,Industrials,Capital Goods,Electrical Equipment,"Preformed Line Products Company, together with..."
1,Health Care,Health Care Equipment & Services,Health Care Equipment & Supplies,"Vapotherm, Inc. is a global medical technology..."
2,Consumer Discretionary,Consumer Services,"Hotels, Restaurants & Leisure",Six Flags Entertainment Corporation is a regio...
3,Materials,Materials,Metals & Mining,"Comstock Inc., formerly Comstock Mining Inc., ..."
4,Industrials,Capital Goods,Aerospace & Defense,"Maxar Technologies, Inc. is a space technology..."
...,...,...,...,...
939,Health Care,Health Care Equipment & Services,Health Care Equipment & Supplies,"Cryoport, Inc. is a provider of integrated tem..."
940,Industrials,Capital Goods,Aerospace & Defense,"Air Industries Group is a holding company, whi..."
941,Financials,Banks,Banks,Huntington Bancshares Incorporated is a divers...
942,Real Estate,Real Estate,Equity Real Estate \nInvestment Trusts \n(REITs),"Empire State Realty Trust, Inc. is a self-admi..."


In [4]:
X

,Sector,Industry Group,Industry
0,Industrials,Commercial & Professional Services,Professional Services
1,Health Care,"Pharmaceuticals, Biotechnology & Life Sciences",Biotechnology
2,Financials,Banks,Banks
3,Health Care,"Pharmaceuticals, Biotechnology & Life Sciences",Biotechnology
4,Consumer Staples,"Food, Beverage & Tobacco",Food Products
...,...,...,...
2195,Health Care,"Pharmaceuticals, Biotechnology & Life Sciences",Biotechnology
2196,Financials,Banks,Banks
2197,Information Technology,Technology Hardware & Equipment,"Electronic Equipment, Instruments & Components"
2198,Financials,Diversified Financials,Capital Markets


In [5]:
le1 = LabelEncoder()
le1.fit(pd.concat([X['Sector'],test_df['Sector']], axis=0))
X['Sector'] = le1.transform(X['Sector'])
le2 = LabelEncoder()
le2.fit(pd.concat([X['Industry Group'],test_df['Industry Group']], axis=0))
X['Industry Group'] = le2.transform(X['Industry Group'])
le3 = LabelEncoder()
le3.fit(pd.concat([X['Industry'],test_df['Industry']], axis=0))
X['Industry'] = le3.transform(X['Industry'])
test_df['Sector'] = le1.transform(test_df['Sector'])
test_df['Industry Group'] = le2.transform(test_df['Industry Group'])
test_df['Industry'] = le3.transform(test_df['Industry'])
X

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

,Sector,Industry Group,Industry
0,6,3,55
1,5,15,7
2,4,1,5
3,5,15,7
4,2,9,28
...,...,...,...
2195,5,15,7
2196,4,1,5
2197,7,20,23
2198,4,6,9


In [6]:
y

0       Human Resource & Employment Services
1                              Biotechnology
2                             Regional Banks
3                              Biotechnology
4                     Packaged Foods & Meats
                        ...                 
2195                           Biotechnology
2196                          Regional Banks
2197                 Technology Distributors
2198        Asset Management & Custody Banks
2199                        Casinos & Gaming
Name: Sub-Industry, Length: 2200, dtype: object

In [7]:
le4 = LabelEncoder()
le4.fit(y)
y = le4.transform(y)
y

array([ 77,  15, 126, ..., 145,  11,  20])

In [8]:
train_sentences = train_df['Description'].to_list()
test_sentences = test_df['Description'].to_list()
vocab_size = 10000
embedding_dim = 32
max_length = 20
trunc_type = 'post'
oov_token = '<OOV>'
tokenizer = keras.preprocessing.text.Tokenizer(num_words = vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index
train_sequences = tokenizer.texts_to_sequences(train_sentences)
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = keras.preprocessing.sequence.pad_sequences(test_sequences, maxlen=max_length)
padded = keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen=max_length)
test_padded

array([[  14,   28,  347, ...,   45,   55,   56],
       [  29,   74,    4, ...,    2,  330,   45],
       [ 124,   20, 1645, ...,  204,   55,   56],
       ...,
       [ 117,   43,    2, ...,  149,   55,   56],
       [   3,  542,  338, ...,  490,  169,  401],
       [ 863,   37, 1692, ...,    4,    3,  162]], dtype=int32)

In [9]:
y.min()

0

In [10]:
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    keras.layers.Flatten(),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(155, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy', optimizer='Adam')
model.fit(padded, y, epochs=20)
X['Description'] = model.predict(padded).argmax(axis=1)
test_df['Description'] = model.predict(test_padded).argmax(axis=-1)

Epoch 1/20
69/69 [==============================] - 1s 5ms/step - loss: 4.7807
Epoch 2/20
69/69 [==============================] - 0s 5ms/step - loss: 3.1521
Epoch 3/20
69/69 [==============================] - 0s 6ms/step - loss: 1.7615
Epoch 4/20
69/69 [==============================] - 1s 11ms/step - loss: 1.0319
Epoch 5/20
69/69 [==============================] - 1s 8ms/step - loss: 0.6544
Epoch 6/20
69/69 [==============================] - 1s 8ms/step - loss: 0.4340
Epoch 7/20
69/69 [==============================] - 1s 9ms/step - loss: 0.2935
Epoch 8/20
69/69 [==============================] - 1s 8ms/step - loss: 0.2035
Epoch 9/20
69/69 [==============================] - 1s 10ms/step - loss: 0.1452
Epoch 10/20
69/69 [==============================] - 1s 8ms/step - loss: 0.1087
Epoch 11/20
69/69 [==============================] - 1s 9ms/step - loss: 0.0856
Epoch 12/20
69/69 [==============================] - 1s 9ms/step - loss: 0.0694
Epoch 13/20
69/69 [============================

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [11]:
y

array([ 77,  15, 126, ..., 145,  11,  20])

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train

,Sector,Industry Group,Industry,Description
2148,1,17,41,90
1787,5,15,7,15
1806,9,16,26,130
1011,8,13,10,23
414,6,2,65,151
...,...,...,...,...
35,5,15,7,15
691,1,4,42,94
1614,6,2,44,2
1222,9,16,26,132


In [13]:
clf = XGBClassifier(max_depth=3, n_estimators=500)
clf.fit(X_train, y_train)

XGBClassifier(n_estimators=500, objective='multi:softprob')

In [14]:
clf.score(X_test, y_test)

0.9840909090909091

In [15]:
y = pd.DataFrame({'Sub-Industry': le4.inverse_transform(clf.predict(test_df))})
y

,Sub-Industry
0,Electrical Components & Equipment
1,Health Care Equipment
2,Leisure Facilities
3,Gold
4,Aerospace & Defense
...,...
939,Health Care Equipment
940,Aerospace & Defense
941,Regional Banks
942,Diversified REITs


In [16]:
y.to_csv('output.csv', index=False,)